In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import matplotlib as mtp
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine

engine = create_engine("postgresql:///kcmo-mc")
db_conn = engine.connect()

In [ ]:
# # code for creating ordvpost2012 table
# 
# drop table if exists clean.ordvpost2012;
# create table clean.ordvpost2012 as (
#     with cases_wzip as (
#     with cases as (
#     select 
#         person_id,
#         disp_date,
#         statute_ord,
#         coalesce (prob_months, 0) as p_months,
#         coalesce(prob_years, 0) as p_years,
#         case_num,
#         sent_exec_code 
#     from clean.dispositions
#    -- where sent_exec_code = 'SES' or sent_exec_code = 'SIS'
#     where disp_date > '2011-12-31'
#     ),
#     wdescr as (
#     select 
#         person_id,
#         cases.case_num,
#         disp_date,
#         p_months,
#         p_years,
#         cf.code,
#         cf.code_desc,
#         p_months + 12 * p_years as prob_total_months,
#         start_dttm::date,
#         final_action_dttm::date,
#         final_action as outcome,
#         statute_ord,
#         sent_exec_code 
#     from cases
#     left join raw_court.dssgc_ordv6222022 ov
#         on ov.case_num = cases.case_num
#     left join raw_court.codes_fixed cf
#     on ov.prob_code = cf.code
#     where code_type = 'PROBCNDS'
#     ),
#     w_rank as (
#     select 
#         wdescr.*,
#         row_number() over(
#             partition by person_id, case_num, code
#             order by final_action_dttm desc,
#             case when final_action_dttm is null then 0 else 1 end,
#             final_action_dttm desc) as rnk
#     from wdescr
#     )
#     select 
#         * 
#     from w_rank 
#     where rnk = 1
#     )
#     select 
#         wz.*,
#         substring(rcda.zipcode, 1, 5) as zipcode, 
#         d.race,
#         d.sex
#     from cases_wzip wz
#     left join raw_court.dssgaddress6222022 rcda 
#         on rcda.case_num = wz.case_num
#     left join raw_court.dssgname d
#         on d.case_num = wz.case_num
# )

In [ ]:
# # code for creating ordvpost2012_nonviol table

# drop table if exists clean.ordvpost2012_nonviol;
# create table clean.ordvpost2012_nonviol as (
#     with cases_wzip as (
#     with cases as (
#     select 
#         person_id,
#         disp_date,
#         statute_ord,
#         coalesce (prob_months, 0) as p_months,
#         coalesce(prob_years, 0) as p_years,
#         case_num,
#         sent_exec_code 
#     from clean.dispositions
#    -- where sent_exec_code = 'SES' or sent_exec_code = 'SIS'
#     where disp_date > '2011-12-31'
#     and statute_ord not in (
#         '50-157(B)(2)',	--'ASSAULT PUBLIC OPERATOR'
#         '50-169',	--'DOMESTIC ASSAULT'
#         '50-170(A)',--'ASSAULT SCHOOL'
#         '50-169',	--'ASSAULT'
#         '50-168',	--'ATTEMPT ASSAULT'
#         '50-231(A)(1)',	--'ENDANGR WELFARE OF CHILD'
#         '50-242',	--'ABUSE OF A CHILD'
#         '50-263',	--'DISCHARGE OF FIREARM'
#         '50-232',	--'SEX ABUSE OF CHILD'
#         '50-157(C)',--'HURL MISSILE AT PUB TRAN'
#         '70-336',	--'FAIL YIELD-EMERGENCY VEH'
#         '70-857B4',	--'VIOL CHILD SAFE BELT LAW'
#         '70-857B3',	--'FAIL SECURE CHILD UNDR 8'
#         '70-857B2',	--'FAIL SECURE CHILD U 40LB'
#         '70-857B1',	--'CHILD RESTRAINED UNDER 4'
#         '70-856C',	--'SEAT BELTS CHILD 4-16YO'
#         '50-261(A)(4)',	--'CARRY WEAPON CHURCH'
#         '50-261(A)(5)',	--'CARRY WEAPON SCHOOL'
#         '50-9',		--'STALKING'
#         '50-244(2)',	--'ALLOW BULLY MINOR'
#         '50-244(1)',	--'BULLY CYBER BULLY MINOR'
#         '50-193(A)',	--'LEAVING EXPLOSIVE DEVICE'
#         '14-37',	--'DISPOSAL OF DEAD ANIMALS'
#         '48-34',	--'RAT INFEST RAT HARBORAGE'
#         '48-50',	--'CHRONIC NUISANCE'
#         '62-89A',	--'ILLEGAL DUMPING'
#         '56-544(A)',	--'DANGEROUS BUILD TICKET 1'
#         '56-544(A)',	--'DANGEROUS BUILD TICKET 2'
#         '56-544(B)',	--'DANGEROUS BUILDINGS'
#         '50-109(D)',	--'LANDLORD RETALIATE'
#         '50-109(E)',	--'LANDLORD NOT ALLOW TERM'
#         '14-31(B)',	--'ANIMAL PUT PERSON FEAR'
#         '14-32',	--'CONFINEMENT-DOG IN HEAT'
#         '14-37(B)',	--'DISPOSAL OF DEAD ANIMALS'
#         '14-47',	--'GUARD DOG'
#         '14-60(D)',	--'PIT BULL BREEDING'
#         '14-60(B)',	--'SPAYING AND NEUTERING'
#         '14-38',	--'ANIMAL MARKETS'
#         '14-41',	--'CONFINEMENT ANIMAL BITE'
#         '14-46',	--'ANIMAL SHOWS'
#         '14-60(G)',	--'PIT BULL SALE-TRANSFER'
#         '14-17',	--'INJ TRAP POISON ANIMAL'
#         '14-15(A)',	--'SM AN OR FOWL PEN'
#         '14-13(A)',	--'VIET POTBELLIED PIGS'
#         '14-9A',	--'DANGEROUS ANIMALS'
#         '14-9B',	--'PROHIBITED ANIMALS'
#         '14-30',	--'EXCESSIVE ANIMAL NOISE'
#         '14-15(D)',	--'SM AN OR FOWL AREA'
#         '50-174(C)(2)',	--'NUISANCE PARTY'
#         '14-15(C)',	--'SM AN OR FOWL WASTE'
#         '14-15(E)',	--'SM AN OR FOWL ENCLOSURE'
#         '14-16(B)',	--'ABUSE OF ANIMAL'
#         '14-15(F)',	--'SM AN OR FOWL NUMBER ROS'
#         '14-61B',	--'POLICE SERVICE ANIMAL'
#         '14-29',	--'DANGEROUS DOGS'
#         '14.16.1',	--'ANIMAL FIGHT'
#         '14-16A',	--'ADEQUATE ANIMAL CARE'
#         '14-48',	--'ANIMAL LEFT IN VEHICLE'
#         '14-16A',	--'ADEQUATE ANIMAL CARE'
#         '14-11',	--'ABANDONMENT'
#         '14-28A'	--'LIMITATION CATS DOGS'
#         )
#     ),
#     wdescr as (
#     select 
#         person_id,
#         cases.case_num,
#         disp_date,
#         p_months,
#         p_years,
#         cf.code,
#         cf.code_desc,
#         p_months + 12 * p_years as prob_total_months,
#         start_dttm::date,
#         final_action_dttm::date,
#         final_action as outcome,
#         statute_ord,
#         sent_exec_code 
#     from cases
#     left join raw_court.dssgc_ordv6222022 ov
#         on ov.case_num = cases.case_num
#     left join raw_court.codes_fixed cf
#     on ov.prob_code = cf.code
#     where code_type = 'PROBCNDS'
#     ),
#     w_rank as (
#     select 
#         wdescr.*,
#         row_number() over(
#             partition by person_id, case_num, code
#             order by final_action_dttm desc,
#             case when final_action_dttm is null then 0 else 1 end,
#             final_action_dttm desc) as rnk
#     from wdescr
#     )
#     select 
#         * 
#     from w_rank 
#     where rnk = 1
#     )
#     select 
#         wz.*,
#         substring(rcda.zipcode, 1, 5) as zipcode, 
#         d.race,
#         d.sex
#     from cases_wzip wz
#     left join raw_court.dssgaddress6222022 rcda 
#         on rcda.case_num = wz.case_num
#     left join raw_court.dssgname d
#         on d.case_num = wz.case_num
# )


In [ ]:
# Counts for all SIS cases
q_SIS_2012 = """ 
select 
    count(distinct person_id) as distinct_indv,
    count(distinct case_num) as distinct_cases,
    count(*) as rows
from clean.ordvpost2012 ov
where sent_exec_code = 'SIS'
"""
SIS_totals = pd.read_sql(q_SIS_2012, db_conn) 

# Counts for nonviolent SIS cases
q_SIS_2012nv = """ 
select 
    count(distinct person_id) as distinct_indv,
    count(distinct case_num) as distinct_cases,
    count(*) as rows
from clean.ordvpost2012_nonviol ov
where sent_exec_code = 'SIS'
"""
SIS_totals_nonviol = pd.read_sql(q_SIS_2012nv, db_conn) 

# Counts for all SES cases
q_SES_2012 = """ 
select 
    count(distinct person_id) as distinct_indv,
    count(distinct case_num) as distinct_cases,
    count(*) as rows
from clean.ordvpost2012 ov
where sent_exec_code = 'SES'
"""
SES_totals = pd.read_sql(q_SES_2012, db_conn) 

# Counts for nonviolent SES cases
q_SES_2012_nv = """ 
select 
    count(distinct person_id) as distinct_indv,
    count(distinct case_num) as distinct_cases,
    count(*) as rows
from clean.ordvpost2012_nonviol ov
where sent_exec_code = 'SES'
"""
SES_totals_nonviol = pd.read_sql(q_SES_2012_nv, db_conn) 

In [ ]:
totals = pd.concat([SIS_totals, SIS_totals_nonviol, SES_totals, SES_totals_nonviol])
totals.index = ['All SIS', 'Nonviolent SIS', 'All SES', 'Nonviolent SES']
totals.columns = ['Number of distinct individuals', 'Number of distinct case numbers', ' Total rows']
totals

Beginning in 2012,

SIS CASES
- 47104 distinct individuals had 74961 cases
- 42165 distinct individuals had 66243 cases excluding cases w/ violent charges 
- 8718 cases with charges in the exclusion list (11.6% of SIS cases)
- 4939 individuals had cases with charges in the exclusion list

SES CASES
- 15187 distinct individuals had 31948 cases
- 13186 distinct individuals had 26380 cases excluding cases w/ violent charges 
- 5568 cases with charges in the exclusion list (17.4% of SES cases)
- 2001 individuals had cases with charges in the exclusion list

In [ ]:
q_siscases = """
select 
*
from clean.ordvpost2012 sp 
where sent_exec_code = 'SIS'
"""
SIScases_prob_term_level = pd.read_sql(q_siscases, db_conn)  
# Replace missing values in the outcome column with the string 'None'
SIScases_prob_term_level['outcome'].fillna('None', inplace = True)

# Add columns for projected end date for probation, and code it as 1 if the most 
# recent disposition date in the dataset has not passed yet
SIScases_prob_term_level['start_dttm']= pd.to_datetime(SIScases_prob_term_level['start_dttm'])
SIScases_prob_term_level['projected_end_dt'] = SIScases_prob_term_level['start_dttm'] + SIScases_prob_term_level['prob_total_months'].astype('timedelta64[M]')
SIScases_prob_term_level['end_date_passed'] = np.where(SIScases_prob_term_level['projected_end_dt'] < max(SIScases_prob_term_level['disp_date']), 1, 0)
SIScases_prob_term_level = SIScases_prob_term_level.drop(['rnk', 'p_months', 'p_years'], axis = 1)
SIScases_prob_term_level.head()

In [ ]:
q_sescases = """
select 
    *
from clean.ordvpost2012 sp 
where sent_exec_code = 'SES'
"""
SEScases_prob_term_level = pd.read_sql(q_sescases, db_conn)  
# Replace missing values in the outcome column with the string 'None'
SEScases_prob_term_level['outcome'].fillna('None', inplace = True)

# Add columns for projected end date for probation, and code it as 1 if the most 
# recent disposition date in the dataset has not passed yet
SEScases_prob_term_level['start_dttm']= pd.to_datetime(SEScases_prob_term_level['start_dttm'])
SEScases_prob_term_level['projected_end_dt'] = SEScases_prob_term_level['start_dttm']+ SEScases_prob_term_level['prob_total_months'].astype('timedelta64[M]')
SEScases_prob_term_level['end_date_passed'] = np.where(SEScases_prob_term_level['projected_end_dt'] < max(SEScases_prob_term_level['disp_date']), 1, 0)
SEScases_prob_term_level = SEScases_prob_term_level.drop(['rnk', 'p_months', 'p_years'], axis = 1)
SEScases_prob_term_level.head()

In [ ]:
# Which 20 probation terms were most commonly assigned to SIS cases after 2011-12-31
# at what rates?
SIS_prob_code_counts = SIScases_prob_term_level.groupby(['code', 'code_desc']).size().reset_index(name='count').sort_values(by='count', ascending=False)
SIS_prob_code_counts['proportion'] = round(SIS_prob_code_counts['count']/SIS_totals['distinct_cases'].values[0], 2)
SIS_prob_code_counts.head(20)

In [ ]:
# Which 20 probation terms were most commonly assigned to SES cases after 2011-12-31
# at what rates?
SES_prob_code_counts = SEScases_prob_term_level.groupby(['code', 'code_desc']).size().reset_index(name='count').sort_values(by='count', ascending=False)
SES_prob_code_counts['proportion'] = round(SES_prob_code_counts['count']/SES_totals['distinct_cases'].values[0], 2)
SES_prob_code_counts.head(20)

In [ ]:
# Filter for cases for which the projected end for the probation passed
SIScases_prob_term_level_end_passed = SIScases_prob_term_level[(SIScases_prob_term_level['end_date_passed']==1)]
# Select relevant columns
SIS_end_date_passed_filter = SIScases_prob_term_level_end_passed[['code', 'code_desc', 'outcome', 'case_num']]
# Count number of cases for each probation term - outcome combination
SIS_end_date_passed_filter = SIS_end_date_passed_filter.groupby(['code', 'code_desc', 'outcome']).size().reset_index(name = 'count')
# Compute proportion
SIS_end_date_passed_filter['proportion'] = round(SIS_end_date_passed_filter['count']/SIS_end_date_passed_filter.groupby('code')['count'].transform('sum'), 3)
# Format columns
SIS_end_date_passed_filter = SIS_end_date_passed_filter.rename(columns = {'code':'probation','code_desc':'description'})
# Replace NAs with "None" to form a category
SIS_end_date_passed_filter['outcome'] = SIS_end_date_passed_filter['outcome'].fillna(value = 'None')
SIS_end_date_passed_filter.sort_values('count', ascending=False)
SIS_end_date_passed_filter

In [ ]:
# Loop over each probation condition
for prob in SIS_end_date_passed_filter['probation'].unique():
    # Crete temp object storing the data for that probation condition
    obj_prob = SIS_end_date_passed_filter[(SIS_end_date_passed_filter['probation']==prob)]

    # For the caption containing outcome counts, create a zipped list of outcomes and counts
    zlist = list(zip(obj_prob['outcome'], obj_prob['count']))
    caption = "" # Initialize empty string
    # For each outcome, count, plug in the appropriate values from the zipped list to form the caption string
    for outcomes, counts in zlist:
        caption = caption + " " + '{}: {}'.format(outcomes, counts)

    # If there is more than one outcome and more than one observation for the probation term;
    if ((obj_prob.shape[0] > 1) and (obj_prob['count'].sum() > 1)):
        plt.clf() # Clear plot space
        # Create barplot of outcome proportions sorted in descending order
        pl_loop = sns.barplot(data = obj_prob.sort_values(by = 'proportion', ascending = False),
                                 x = 'outcome', y = 'proportion', color = 'lightblue')
        pl_loop.set_title("Projected date outcomes for SIS cases receiving probation type \n{}\n{}".format(prob, obj_prob['description'].values[0]))
        plt.ylim(0,1)
        # Show the proportion on each bar
        pl_loop.bar_label(pl_loop.containers[0])
        # Add caption to plot
        pl_loop.text(x = -0.9, y = -0.25, s = " Outcome counts:")
        pl_loop.text(x = -0.9, y = -0.31, s = caption)
        pl_loop.text(x = -0.9, y = -0.425, s = " Total count: " + str(obj_prob['count'].sum()))
        # Save plot with given name and format specifications
        plt.savefig('SIS_post2011_{}_probation_outcomes.png'.format(prob),  dpi = 300, bbox_inches = "tight")
        plt.show()

In [ ]:
# Filter for cases for which the projected end for the probation passed
SEScases_prob_term_level_end_passed = SEScases_prob_term_level[(SEScases_prob_term_level['end_date_passed']==1)]
# Select relevant columns
SES_end_date_passed_filter = SEScases_prob_term_level_end_passed[['code', 'code_desc', 'outcome', 'case_num']]
# Count number of cases for each probation term - outcome combination
SES_end_date_passed_filter = SES_end_date_passed_filter.groupby(['code', 'code_desc', 'outcome']).size().reset_index(name = 'count')
# Compute proportion
SES_end_date_passed_filter['proportion'] = round(SES_end_date_passed_filter['count']/SES_end_date_passed_filter.groupby('code')['count'].transform('sum'), 3)
# Format columns
SES_end_date_passed_filter = SES_end_date_passed_filter.rename(columns = {'code':'probation','code_desc':'description'})
# Replace NAs with "None" to form a category
SES_end_date_passed_filter['outcome'] = SES_end_date_passed_filter['outcome'].fillna(value = 'None')
SES_end_date_passed_filter.sort_values('count', ascending=False)

In [ ]:
# Loop over each probation condition
for prob in SES_end_date_passed_filter['probation'].unique():
    # Crete temp object storing the data for that probation condition
    obj_prob = SES_end_date_passed_filter[(SES_end_date_passed_filter['probation']==prob)]

    # For the caption containing outcome counts, create a zipped list of outcomes and counts
    zlist = list(zip(obj_prob['outcome'], obj_prob['count']))
    caption = "" # Initialize empty string
    # For each outcome, count, plug in the appropriate values from the zipped list to form the caption string
    for outcomes, counts in zlist:
        caption = caption + " " + '{}: {}'.format(outcomes, counts)

    # If there is more than one outcome and more than one observation for the probation term;
    if ((obj_prob.shape[0] > 1) and (obj_prob['count'].sum() > 1)):
        plt.clf() # Clear plot space
        # Create barplot of outcome proportions sorted in descending order
        pl_loop = sns.barplot(data = obj_prob.sort_values(by = 'proportion', ascending = False),
                                 x = 'outcome', y = 'proportion', color = 'steelblue')
        pl_loop.set_title("Projected date outcomes for SES cases receiving probation type \n{}\n{}".format(prob, obj_prob['description'].values[0]))
        plt.ylim(0,1)
        # Show the proportion on each bar
        pl_loop.bar_label(pl_loop.containers[0])
        # Add caption to plot
        pl_loop.text(x = -0.9, y = -0.25, s = " Outcome counts:")
        pl_loop.text(x = -0.9, y = -0.31, s = caption)
        pl_loop.text(x = -0.9, y = -0.425, s = " Total count: " + str(obj_prob['count'].sum()))
        # Save plot with given name and format specifications
        plt.savefig('SES_post2011_{}_probation_outcomes.png'.format(prob),  dpi = 300, bbox_inches = "tight")
        plt.show()

In [ ]:
# Compute the proportions of different community service outcomes for SIS cases
CS_SIS_end_date_passed = SIS_end_date_passed_filter[SIS_end_date_passed_filter.probation.str.contains('CMS') | SIS_end_date_passed_filter.probation.str.contains('COMSRV')  ]
CS_SIS_end_date_passed.sort_values('count', ascending=False)

In [ ]:
# Compute the proportions of different community service outcomes for SES cases
CS_SES_end_date_passed = SES_end_date_passed_filter[SES_end_date_passed_filter.probation.str.contains('CMS') | SES_end_date_passed_filter.probation.str.contains('COMSRV')  ]
CS_SES_end_date_passed.sort_values('count', ascending = False)

In [ ]:
warnings.filterwarnings('ignore')
# Loop over each outcome
for outcome in CS_SIS_end_date_passed['outcome'].unique():
    plt.clf()
    dat = CS_SIS_end_date_passed[CS_SIS_end_date_passed['outcome']==outcome]

   # For the caption containing outcome counts, create a zipped list of outcomes and counts
    zlist = list(zip(dat['probation'], dat['count']))
    caption = "Outcome counts:\n" # Initialize  string
    xlabs = list()
    counter = 0
    for probs, counts in zlist:
    # For each probation, count, plug in the appropriate values from the zipped list to form the caption string
        probs = probs.replace("CMSR", "").replace("CMS", "").replace("COMSRV", "999")
        if probs != '999':
            caption = caption + " " + '{}-hour: {}'.format(probs, counts)
        if probs == '999':
            caption = caption + " " + '-hour: {}'.format(counts)
        if counter in [4, 7]:
            caption = caption + "\n"
        xlabs.append(int(probs)) 
        counter = counter + 1
    dat['probation'] =  xlabs
    dat = dat.sort_values(by ='probation')
    
    pl = sns.barplot(data = dat, x = 'probation', y = 'proportion', color = 'salmon')
    plt.ylim(0, 1)
    pl.bar_label(pl.containers[0])  # Show the proportion on each bar
    pl.set_title("Community Service outcome: {} rates".format(outcome))
    xlabs.sort()
    xlabs[-1] = '-'
    pl.set_xticklabels(xlabs)
    pl.set_xlabel('hours')
    pl.text(x = -0.9, y = -0.31, s = caption)
    pl.text(x = -0.9, y = -0.425, s = " Total count: " + str(dat['count'].sum()))
    plt.show()

In [ ]:
# Community service outcomes for SES
# Loop over each outcome
for outcome in CS_SES_end_date_passed['outcome'].unique():
    plt.clf()
    dat = CS_SES_end_date_passed[CS_SES_end_date_passed['outcome']==outcome]

   # For the caption containing outcome counts, create a zipped list of outcomes and counts
    zlist = list(zip(dat['probation'], dat['count']))
    caption = "Outcome counts:\n" # Initialize  string
    xlabs = list()
    counter = 0
    for probs, counts in zlist:
    # For each probation, count, plug in the appropriate values from the zipped list to form the caption string
        probs = probs.replace("CMSR", "").replace("CMS", "").replace("COMSRV", "999")
        if probs != '999':
            caption = caption + " " + '{}-hour: {}'.format(probs, counts)
        if probs == '999':
            caption = caption + " " + '-hour: {}'.format(counts)
        if counter in [4, 7]:
            caption = caption + "\n"
        xlabs.append(int(probs)) 
        counter = counter + 1
    dat['probation'] =  xlabs
    dat = dat.sort_values(by ='probation')
    
    pl = sns.barplot(data = dat, x = 'probation', y = 'proportion', color = 'indianred')
    plt.ylim(0, 1)
    pl.bar_label(pl.containers[0])  # Show the proportion on each bar
    pl.set_title("SES Community Service outcome: {} rates".format(outcome))
    xlabs.sort()
    xlabs[-1] = '-'
    pl.set_xticklabels(xlabs)
    pl.set_xlabel('hours')
    pl.text(x = -0.9, y = -0.31, s = caption)
    pl.text(x = -0.9, y = -0.425, s = " Total count: " + str(dat['count'].sum()))
    plt.show()

In [ ]:
SIScases_prob_term_level.groupby(['sent_exec_code', 'sex']).count()

In [ ]:
# Create df with person id, case num, disp date, and the total number of 
# conditions associated with each person - case number combination
SIScases_case_level = SIScases_prob_term_level.groupby(['person_id', 'case_num', 'disp_date']).size().reset_index(name='num_total_terms')
SIScases_case_level

In [ ]:
# compute the total number of cases for which the projected end date has not passed
SIScases_inprogress_counts = SIScases_prob_term_level[SIScases_prob_term_level['end_date_passed']==0].groupby(['person_id', 'case_num', 'disp_date']).size().reset_index(name='num_term_in_progress')

# add a column for the number of cases for which the projected end date has not passed
SIScases_case_level_prog = SIScases_case_level.merge(SIScases_inprogress_counts, on = ['person_id','case_num', 'disp_date'], how = 'left')
SIScases_case_level_prog['num_term_in_progress'] = SIScases_case_level_prog['num_term_in_progress'].fillna(0)

In [ ]:
# Create a df with person id, case num, disp date, probation condition 
# outcome, and the number of completed conditions associated with the case
SIScases_prob_term_level_comp_counts = SIScases_prob_term_level[SIScases_prob_term_level['outcome']=='CMPL'].groupby(['person_id', 'case_num', 'disp_date', 'outcome']).size().reset_index(name = 'num_term_complete') 
#SIScases_prob_term_level_comp_counts

In [ ]:
# Create df with person_id, case_num, disp_date, number of total cases in the database,
# number of cases currently in progress, number of 
SIScases_case_level2 = SIScases_case_level_prog.merge(SIScases_prob_term_level_comp_counts, on = ['person_id','case_num', 'disp_date'], how = 'outer')
SIScases_case_level2['num_term_complete'] = SIScases_case_level2['num_term_complete'].fillna(0)
SIScases_case_level2 = SIScases_case_level2.drop('outcome', axis = 1)
SIScases_case_level2

In [ ]:
# Create a column indicating whether the individual has completed all conditions
# associated with a given case
SIScases_case_level2['all_cond_complete_or_in_prog'] = np.where((SIScases_case_level2['num_term_complete'] + SIScases_case_level2['num_term_in_progress'])==SIScases_case_level2['num_total_terms'], 1, 0)
SIScases_case_level2['all_cond_complete'] = np.where((SIScases_case_level2['num_total_terms'])==SIScases_case_level2['num_term_complete'], 1, 0)
SIScases_case_level2

In [ ]:
SIScases_case_level2['all_cond_complete'].sum()

In [ ]:
outcome_dist = SIScases_prob_term_level.value_counts('outcome')
outcome_dist

In [ ]:
outcome_dist.sum()

In [ ]:
SIScases_prob_term_level.value_counts('end_date_passed')

In [ ]:
SIScases_prob_term_level.value_counts('code').head(10)

In [ ]:
SIScases_prob_term_level.value_counts('prob_total_months')

In [ ]:
q_cases = """ 
select 
    person_id,
    count(distinct case_num) as num_cases
from clean.ordvpost2012
where sent_exec_code = 'SIS'
group by person_id
"""
cases = pd.read_sql(q_cases, db_conn)  
cases['curr_num_successful_cases'] = SIScases_case_level2.groupby('person_id')['all_cond_complete_or_in_prog'].sum().values
cases['num_successful_cases'] = SIScases_case_level2.groupby('person_id')['all_cond_complete'].sum().values
cases['curr_successful_defendant'] = np.where(cases['num_cases']==cases['curr_num_successful_cases'], 1, 0)
cases['all_successful_defendant'] = np.where(cases['num_cases']==cases['num_successful_cases'], 1, 0)
cases

# num cases is the total number of cases belonging to a defendant
# curr_num_successful_cases is the number of cases that are complete
#        or currently in progress (so currently successful cases; we don't know the outcome
#  for the in-progress cases)
# num_successful_cases is the number of cases that are complete by their projected end dates
# curr_successful_defendant is 1 if all cases belonging to a defendant had all terms completed
#  or cases are currently in progress.
# all_successful_defendant is 1 if all cases belonging to a defendant had all terms completed 

In [ ]:
pd.DataFrame(data=cases,columns=['num_cases','num_successful_cases', 'curr_num_successful_cases', 'all_successful_defendant', 'curr_successful_defendant']).sum()


Based on the table above, 25,884 SIS probationers completed all conditions for all their cases and 1,076 have not yet passed the projected end date (the probationary term is currently in progress).

--numbers need to be updated

After 2011, 47104 unique individuals had 74961 total SIS cases.

36889 out of 74961 total cases had each probation condition associated with the case completed (approximately 49% of cases).

Out of the 47104 unique individuals, only 25884 completed all probation conditions for each of their cases (approximately 50%).

In [ ]:
# # Loop over each probation condition
# for prob in SIS_end_date_passed_filter['probation'].unique():
#     # Crete temp object storing the data for that probation condition
#     obj_prob = SIS_end_date_passed_filter[(SIS_end_date_passed_filter['probation']==prob)]

#     # For the caption containing outcome counts, create a zipped list of outcomes and counts
#     zlist = list(zip(obj_prob['outcome'], obj_prob['count']))
#     caption = "" # Initialize empty string
#     # For each outcome, count, plug in the appropriate values from the zipped list to form the caption string
#     for outcomes, counts in zlist:
#         caption = caption + " " + '{}: {}'.format(outcomes, counts)

#     # If there is more than one outcome and more than one observation for the probation term;
#     if ((obj_prob.shape[0] > 1) and (obj_prob['count'].sum() > 1)):
#         plt.clf() # Clear plot space
#         # Create barplot of outcome proportions sorted in descending order
#         pl_loop = sns.barplot(data = obj_prob.sort_values(by = 'proportion', ascending = False),
#                                  x = 'outcome', y = 'proportion', color = 'lightblue')
#         pl_loop.set_title("Projected date outcomes for SIS cases receiving probation type \n{}\n{}".format(prob, obj_prob['description'].values[0]))
#         plt.ylim(0,1)
#         # Show the proportion on each bar
#         pl_loop.bar_label(pl_loop.containers[0])
#         # Add caption to plot
#         pl_loop.text(x = -0.9, y = -0.25, s = " Outcome counts:")
#         pl_loop.text(x = -0.9, y = -0.31, s = caption)
#         pl_loop.text(x = -0.9, y = -0.425, s = " Total count: " + str(obj_prob['count'].sum()))
#         # Save plot with given name and format specifications
#         plt.savefig('SIS_post2011_{}_probation_outcomes.png'.format(prob),  dpi = 300, bbox_inches = "tight")
#         plt.show()

In [ ]:
# # Filter for cases for which the projected end for the probation passed
# SEScases_prob_term_level_end_passed = SEScases_prob_term_level[(SEScases_prob_term_level['end_date_passed']==1)]
# # Select relevant columns
# SES_end_date_passed_filter = SEScases_prob_term_level_end_passed[['code', 'code_desc', 'outcome', 'case_num']]
# # Count number of cases for each probation term - outcome combination
# SES_end_date_passed_filter = SES_end_date_passed_filter.groupby(['code', 'code_desc', 'outcome']).size().reset_index(name = 'count')
# # Compute proportion
# SES_end_date_passed_filter['proportion'] = round(SES_end_date_passed_filter['count']/SES_end_date_passed_filter.groupby('code')['count'].transform('sum'), 3)
# # Format columns
# SES_end_date_passed_filter = SES_end_date_passed_filter.rename(columns = {'code':'probation','code_desc':'description'})
# # Replace NAs with "None" to form a category
# SES_end_date_passed_filter['outcome'] = SES_end_date_passed_filter['outcome'].fillna(value = 'None')
# SES_end_date_passed_filter

In [ ]:
# # Loop over each probation condition
# for prob in SES_end_date_passed_filter['probation'].unique():
#     # Crete temp object storing the data for that probation condition
#     obj_prob = SES_end_date_passed_filter[(SES_end_date_passed_filter['probation']==prob)]

#     # For the caption containing outcome counts, create a zipped list of outcomes and counts
#     zlist = list(zip(obj_prob['outcome'], obj_prob['count']))
#     caption = "" # Initialize empty string
#     # For each outcome, count, plug in the appropriate values from the zipped list to form the caption string
#     for outcomes, counts in zlist:
#         caption = caption + " " + '{}: {}'.format(outcomes, counts)

#     # If there is more than one outcome and more than one observation for the probation term;
#     if ((obj_prob.shape[0] > 1) and (obj_prob['count'].sum() > 1)):
#         plt.clf() # Clear plot space
#         # Create barplot of outcome proportions sorted in descending order
#         pl_loop = sns.barplot(data = obj_prob.sort_values(by = 'proportion', ascending = False),
#                                  x = 'outcome', y = 'proportion', color = 'steelblue')
#         pl_loop.set_title("Projected date outcomes for SES cases receiving probation type \n{}\n{}".format(prob, obj_prob['description'].values[0]))
#         plt.ylim(0, 1)
#         # Show the proportion on each bar
#         pl_loop.bar_label(pl_loop.containers[0])
#         # Add caption to plot
#         pl_loop.text(x = -0.9, y = -0.25, s = " Outcome counts:")
#         pl_loop.text(x = -0.9, y = -0.31, s = caption)
#         pl_loop.text(x = -0.9, y = -0.425, s = " Total count: " + str(obj_prob['count'].sum()))
#         # Save plot with given name and format specifications
#         plt.savefig('SES_post2011_{}_probation_outcomes.png'.format(prob),  dpi = 300, bbox_inches = "tight")
#         plt.show()

In [ ]:
# counts of outcomes for each probation condition
q_sis_outcome_counts_by_prob_race = """ 
select
    code as probation,
    code_desc as description,
    outcome,
    race,
    count(distinct case_num)
from clean.ordvpost2012
group by code, code_desc, outcome, race
"""
sis_outcome_counts_by_prob_race = pd.read_sql(q_sis_outcome_counts_by_prob_race, db_conn) 
sis_outcome_counts_by_prob_race['proportion'] = round(sis_outcome_counts_by_prob_race['count']/sis_outcome_counts_by_prob_race.groupby('probation')['count'].transform('sum'), 3)
sis_outcome_counts_by_prob_race

In [ ]:
sis_outcome_counts_by_prob_race['race'].unique()

In [ ]:
sis_outcome_counts_by_prob_race.loc[sis_outcome_counts_by_prob_race.loc[:,'race'].isnull(),'race'] = 'U'
sis_outcome_counts_by_prob_race.loc[sis_outcome_counts_by_prob_race.loc[:,'outcome'].isnull(),'outcome'] = 'None'
sis_outcome_counts_by_prob_race['race'] = sis_outcome_counts_by_prob_race['race'].str.strip()
# write SIS probation condition outcomes with race breakdown to text file
sis_outcome_counts_by_prob_race.to_string('sis_outcome_counts_by_prob_race.txt', index = False)

In [ ]:
anh = sis_outcome_counts_by_prob_race[sis_outcome_counts_by_prob_race['probation']=='ANH2O']

In [ ]:
anh

In [ ]:
sns.barplot(x = 'outcome', y='proportion', hue = 'race', data=anh, dodge = False)

In [ ]:
#plt.plot(x= anh.outcome, y= anh.proportion, data = anh, kind='bar', stacked = True)

In [ ]:
# make an arbitrary-length colormap
cm = plt.get_cmap('vlag')
c = [cm(1.0 * i/len(sis_outcome_counts_by_prob_race['race'].unique())) for i in range(len(sis_outcome_counts_by_prob_race['race'].unique()))]
clist2 = {i:j for i, j in zip(sis_outcome_counts_by_prob_race['race'].values, c)}

for prob in sis_outcome_counts_by_prob_race['probation'].unique()[0:9]:
    obj_prob = sis_outcome_counts_by_prob_race[(sis_outcome_counts_by_prob_race['probation']==prob)]
    obj_prob1 = obj_prob.drop(['count'], axis =  1)
    obj_prob2 = obj_prob1
    obj_prob1 = obj_prob1.groupby(['probation', 'description', 'outcome', 'race'], as_index = False).proportion.sum()
    obj_prob2 = obj_prob2.groupby(['probation', 'race'], as_index = False).proportion.sum()
    
    # For the caption containing outcome counts, create a zipped list of outcomes and counts
    zlist = list(zip(obj_prob2['race'], round(obj_prob2['proportion'], 3)))
    
    caption = "" # Initialize empty string
    # For each outcome, count, plug in the appropriate values from the zipped list to form the caption string
    for outcomes, counts in zlist:
        caption = caption + " " + '{}: {}'.format(outcomes, counts)
    
    #print( [clist2[i] for i in  sis_outcome_counts_by_prob_race['race'].values])
    if ((obj_prob.shape[0] > 1) and (obj_prob['count'].sum() > 1)):
        plt.clf() #clear the plot area
        pivoted = pd.pivot(obj_prob1, columns = ['race'], index= ['outcome'], values = ['proportion'])

        pl_loop = pivoted.plot(kind = "bar", stacked = True, color = [clist2[i] for i in  obj_prob1['race'].unique()])
        pl_loop.set_xticklabels(pl_loop.get_xticklabels(), rotation = 90)
        pl_loop.set_title("Racial distribution within outcome types for probation term \n{} ({})".format(prob, obj_prob['description'].values[0]))
        plt.ylim(0, 1)
        # Add caption to plot
        #pl_loop.bar_label(pl_loop.containers) 
        pl_loop.text(x = -0.9, y = -0.45, s = "Overall distribution of race")
        pl_loop.text(x = -0.9, y = -0.5, s = caption)
        pl_loop.text(x = -0.9, y = -0.35, s = "Total number of individuals receiving probation term {}: {}".format(prob, obj_prob['count'].sum()))
        plt.legend(title = 'Race',labels = obj_prob1['race'].unique())
        plt.savefig('SIS_post2011_{}_p_outcomes_race.png'.format(prob), dpi=300, bbox_inches="tight")
        plt.show()

In [ ]:
sis_outcome_counts_by_prob_race[sis_outcome_counts_by_prob_race['probation']=='ANAUTO']['outcome']